In [4]:
from datetime import datetime
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import yfinance as yf

In [13]:
tickers = ['VGIR11.SA', 'BTCI11.SA']

sec_data = pd.DataFrame()

for t in tickers:
    data = yf.download(t, start='2023-01-01', end=datetime.now())
    sec_data[t] = data['Adj Close']  # Assign the 'Adj Close' column directly


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [14]:
sec_data.tail()

,VGIR11.SA,BTCI11.SA
Date,,
2024-02-22,9.78,10.28
2024-02-23,9.77,10.26
2024-02-26,9.61,10.10
2024-02-27,9.68,10.14
2024-02-28,9.65,10.02


In [15]:
sec_returns = np.log(sec_data / sec_data.shift(1))

In [16]:
sec_returns

,VGIR11.SA,BTCI11.SA
Date,,
2023-01-02,NaN,NaN
2023-01-03,0.000000,-0.010841
2023-01-04,-0.004061,-0.007993
2023-01-05,-0.006122,-0.006565
2023-01-06,0.006122,0.001614
...,...,...
2024-02-22,0.001023,0.003899
2024-02-23,-0.001023,-0.001947
2024-02-26,-0.016512,-0.015717


## Active 1

In [22]:
for t in tickers:
    print(f'{t} mean: {sec_returns[t].mean()}')
    print(f'{t} mean annual: {sec_returns[t].mean() * 250}')
    print(f'{t} std: {sec_returns[t].std()}')
    print(f'{t} std annual: {sec_returns[t].std() * 250 ** 0.5}')
    print('\n')

VGIR11.SA mean: 0.0004819514913063621
VGIR11.SA mean annual: 0.12048787282659053
VGIR11.SA std: 0.006166739767424743
VGIR11.SA std annual: 0.09750471701299605


BTCI11.SA mean: 0.0008350097671491719
BTCI11.SA mean annual: 0.208752441787293
BTCI11.SA std: 0.00699492513532259
BTCI11.SA std annual: 0.11059947744990452




***

In [29]:
sec_returns[tickers].mean()

VGIR11.SA    0.000482
BTCI11.SA    0.000835
dtype: float64

In [33]:
sec_returns[tickers].mean() * 250

VGIR11.SA    0.120488
BTCI11.SA    0.208752
dtype: float64

In [34]:
sec_returns[tickers].std() * 250 ** 0.5

VGIR11.SA    0.097505
BTCI11.SA    0.110599
dtype: float64

## Covariance and Correlation

\begin{eqnarray*}
Covariance Matrix: \  \   
\Sigma = \begin{bmatrix}
        \sigma_{1}^2 \ \sigma_{12} \ \dots \ \sigma_{1I} \\
        \sigma_{21} \ \sigma_{2}^2 \ \dots \ \sigma_{2I} \\
        \vdots \ \vdots \ \ddots \ \vdots \\
        \sigma_{I1} \ \sigma_{I2} \ \dots \ \sigma_{I}^2
    \end{bmatrix}
\end{eqnarray*}

In [36]:
VGIR11_var = sec_returns['VGIR11.SA'].var() 
VGIR11_var

3.802867935913777e-05

In [37]:
BTCI11_var = sec_returns['BTCI11.SA'].var() 
BTCI11_var

4.892897764876775e-05

In [38]:
VGIR11_var_a = sec_returns['VGIR11.SA'].var() * 250
VGIR11_var_a

0.009507169839784443

In [39]:
BTCI11_var_a = sec_returns['BTCI11.SA'].var() * 250
BTCI11_var_a

0.012232244412191938

***

In [40]:
cov_matrix = sec_returns.cov()
cov_matrix

,VGIR11.SA,BTCI11.SA
VGIR11.SA,0.000038,0.000006
BTCI11.SA,0.000006,0.000049


In [44]:
cov_matrix_a = sec_returns.cov() * 250
cov_matrix_a

,VGIR11.SA,BTCI11.SA
VGIR11.SA,0.009507,0.001479
BTCI11.SA,0.001479,0.012232


In [47]:
corr_matrix = sec_returns.corr()

In [48]:
corr_matrix

,VGIR11.SA,BTCI11.SA
VGIR11.SA,1.000000,0.137105
BTCI11.SA,0.137105,1.000000


## Calculating Portfolio Risk

Equal weigthing scheme:

In [49]:
weights = np.array([0.5, 0.5])

Portfolio Variance:

In [50]:
pfolio_var = np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))

0.0061741209146871125

In [52]:
pfolio_var

0.0061741209146871125

Portfolio Volatility:

In [53]:
pfolio_vol = (np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))) ** 0.5

In [54]:
pfolio_vol

0.07857557454251997

In [57]:
round(pfolio_vol, 5) * 100

7.858

## Calculating Diversifiable and Non-Diversifiable Risk of a Portfolio

In [58]:
weights = np.array([0.5, 0.5])

In [59]:
weights[0]

0.5

In [60]:
weights[1]

0.5

Diversifiable Risk:

In [70]:
VGIR11_var_a = sec_returns['VGIR11.SA'].var() * 250

In [77]:
VGIR11_var_a

0.009507169839784443

In [72]:
BTCI11_var_a = sec_returns['BTCI11.SA'].var() * 250

In [73]:
BTCI11_var_a

0.012232244412191938

In [74]:
dr = pfolio_var - (weights[0] ** 2 * VGIR11_var_a) - (weights[1] ** 2 * BTCI11_var_a)

In [75]:
dr

0.0007392673516930172

In [76]:
round(dr*100, 3)

0.074

Non-Diversifiable Risk:

In [78]:
n_dr_1 = pfolio_var - dr

0.005434853562994096

In [80]:
n_dr_1

0.005434853562994096

In [81]:
n_dr_2 = (weights[0] ** 2 * VGIR11_var_a) + (weights[1] ** 2 * BTCI11_var_a)

In [82]:
n_dr_2

0.005434853562994096

In [83]:
n_dr_1 == n_dr_2

True